1. How many transactions were completed during each marketing campaign?

In [3]:
select campaign_name, count(transaction_id) as total_transactions
from transactions t 
join marketing_campaigns mc on t.product_id = mc.product_id
where t.purchase_date between mc.start_date and mc.end_date
group by campaign_name
order by total_transactions;

(3 rows affected)

Total execution time: 00:00:00.023

campaign_name,total_transactions
Super Save,1
New Collection Launch,4
Summer Sale,5


2\. Which product had the highest sales quantity?

In [4]:
select top 1 sc.product_name as product_name, sum(quantity) as sales_quantity
from transactions t 
join sustainable_clothing sc on t.product_id = sc.product_id
group by product_name
order by sales_quantity desc;

(1 row affected)

Total execution time: 00:00:00.017

product_name,sales_quantity
Organic Cotton Sweater,9


3\. What is the total revenue generated from each marketing campaign?

In [5]:
select mc.campaign_name as campaign_name, sum(t.quantity * sc.price) as total_revenue 
from transactions t  
join sustainable_clothing sc on t.product_id = sc.product_id
join marketing_campaigns mc on t.product_id = mc.product_id and mc.product_id = t.product_id
group by mc.campaign_name
order by total_revenue;


(3 rows affected)

Total execution time: 00:00:00.026

campaign_name,total_revenue
Super Save,209.96999999999997
New Collection Launch,439.92
Summer Sale,639.92


4\. What is the top-selling product category based on the total revenue generated?

In [6]:
select top 1 sc.category as category, sum(t.quantity * sc.price) as total_revenue 
from transactions t  
join sustainable_clothing sc on t.product_id = sc.product_id
group by sc.category
order by total_revenue desc;

(1 row affected)

Total execution time: 00:00:00.050

category,total_revenue
Bottoms,1289.79


5\. Which products had a higher quantity sold compared to the average quantity sold?

In [7]:
select sc.product_name, sum(t.quantity) as quantity_sold
from transactions t 
join sustainable_clothing sc on t.product_id = sc.product_id
group by sc.product_name
having sum(t.quantity) > avg(t.quantity)
order by quantity_sold desc;


(18 rows affected)

Total execution time: 00:00:00.012

product_name,quantity_sold
Organic Cotton Sweater,9
Recycled Denim Jeans,8
Bamboo Yoga Leggings,8
Linen Jumpsuit,7
Organic Cotton Socks,7
Organic Cotton Skirt,5
Bamboo Lounge Pants,5
Eco-Friendly Hoodie,5
Hemp Baseball Cap,5
Cork Sandals,4


6\. What is the average revenue generated per day during the marketing campaigns?

In [8]:
select mc.campaign_name as campaign_name, 
       sum(t.quantity * sc.price) / (datediff(day, mc.start_date, mc.end_date) + 1) as average_revenue_per_day
from marketing_campaigns mc
join transactions t on mc.product_id = t.product_id
join sustainable_clothing sc on t.product_id = sc.product_id
group by mc.campaign_name, mc.start_date, mc.end_date;

(3 rows affected)

Total execution time: 00:00:00.083

campaign_name,average_revenue_per_day
New Collection Launch,13.7475
Summer Sale,21.330666666666666
Super Save,7.7766666666666655


7\. What is the percentage contribution of each product to the total revenue?

In [9]:
select sc.product_name as product_name, 
       (sum(t.quantity * sc.price) / sum(sum(t.quantity * sc.price)) over ()) * 100 as revenue_percentage
from transactions t
join sustainable_clothing sc on t.product_id = sc.product_id
group by sc.product_name
order by revenue_percentage desc;


(20 rows affected)

Total execution time: 00:00:00.026

product_name,revenue_percentage
Recycled Denim Jeans,13.705366321705162
Linen Jumpsuit,10.49298368857515
Organic Cotton Sweater,9.635862860667538
Bamboo Yoga Leggings,9.421903913371258
Recycled Polyester Jacket,7.709375642519363
Eco-Friendly Hoodie,6.4241227468987745
Bamboo Lounge Pants,5.353257144815298
Upcycled Denim Jacket,5.139512370639436
Hemp Overalls,4.8182526900143925
Organic Cotton Dress,4.49699300938935


8\. Compare the average quantity sold during marketing campaigns to outside the marketing campaigns

In [10]:
select transaction_type, avg(average_quantity) as average_quantity
from (select 
         case when mc.campaign_id is not null then 'marketing campaigns' else 'outside marketing campaigns' 
		 end as transaction_type,
         t.quantity as average_quantity
      from transactions t
      left join marketing_campaigns mc on t.product_id = mc.product_id) subquery
group by transaction_type;

(2 rows affected)

Total execution time: 00:00:00.009

transaction_type,average_quantity
marketing campaigns,1
outside marketing campaigns,1


9\. Compare the revenue generated by products inside the marketing campaigns to outside the campaigns

In [11]:
select transaction_type, sum(revenue) as total_revenue
from (select 
         case when mc.campaign_id is not null then 'marketing campaigns' else 'outside marketing campaigns' 
		 end as transaction_type,
         t.quantity * sc.price as revenue
      from transactions t
      left join marketing_campaigns mc on t.product_id = mc.product_id
      join sustainable_clothing sc on t.product_id = sc.product_id
) subquery
group by transaction_type;

(2 rows affected)

Total execution time: 00:00:00.008

transaction_type,total_revenue
marketing campaigns,1289.81
outside marketing campaigns,3379.3099999999977


10\. Rank the products by their average daily quantity sold

In [12]:
with avg_daily_quantity as (
    select
        sc.product_name as product_name,
        avg(t.quantity * 1.0 / datediff(day, mc.start_date, mc.end_date)) as avg_daily_quantity_sold
    from sustainable_clothing sc
    join transactions t on t.product_id = sc.product_id
    join marketing_campaigns mc on mc.product_id = sc.product_id
    group by sc.product_name)
select
    product_name,
    avg_daily_quantity_sold,
    rank() over (order by avg_daily_quantity_sold desc) as ranking
from avg_daily_quantity
order by ranking;

(3 rows affected)

Total execution time: 00:00:00.046

product_name,avg_daily_quantity_sold,ranking
Bamboo Yoga Leggings,0.043010752688,1
Recycled Denim Jeans,0.039408866994,2
Organic Cotton Dress,0.038461538461,3
